# ファインチューニング 演習

本日の演習ではファインチューニングについて取り扱う。

ファインチューニングは、特定の（大規模）データセットで学習した学習済みモデルを初期値として、他の学習データを用いて学習させることである。

一般に目的のデータ数が少ないとき、大きいモデルを一から学習して精度を高めるのは困難である。そのような状況においてファインチューニングを用いることで、大規模データセットで事前学習された特徴量を初期値として使うことが出来るので、精度を高めることが可能となる。

現に、今日のCIFAR10等のベンチマークの最高精度は、「より大規模なデータセットを用いた事前学習」＋「CIFAR10等によるファインチューニング」という形で達成されている。



本日の演習では、「ImageNetで事前学習されたモデル」と「CIFAR10（の一部）」を用いてファインチューニングを実装し、事前学習しないモデルを用いた場合と比較することでファインチューニングの効果を確認する。

In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
import numpy as np
import tqdm

## 前処理・データローダーの実装

pytorchではImageNetで事前学習されたモデル（学習済みモデル）をダウンロードして用いることが出来る。

事前学習されたモデルを用いる場合、元データと同じ前処理を用いる必要があるので、ここで定義する。

また、CIFARデータローダーも定義する。ファインチューニングの効果をより明確に示すために、ここではCIFAR10の訓練データ数をデフォルトの50000から、10分の1の5000に減らしている。

In [ ]:
# 学習済みモデルでは、前処理に以下の平均分散を用いた正規化を行う
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# データセットの定義に用いるtransformを定義する
# Big Transfer[1] を参考にサイズを128にリサイズする（もともとは32）
transform_valid = transforms.Compose([
                    transforms.Resize(128),
                    transforms.ToTensor(),
                    normalize
                ])

# 訓練データではデータAugmentaionも加える
transform_train = transforms.Compose([
                    transforms.Resize(160),
                    transforms.RandomCrop(128),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.ToTensor(),
                    normalize
                ])


batch_size = 128

# CIFAR10の50000枚の訓練データを分割し、5000枚のみを用いる
train_dataset = datasets.CIFAR10('./data/cifar10', train=True, download=True, transform=transform_train)
train_dataset, _ = torch.utils.data.random_split(train_dataset, [5000, 45000])

dataloader_train = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

valid_dataset = datasets.CIFAR10('./data/cifar10', train=False, download=True, transform=transform_train)
valid_dataset, _ = torch.utils.data.random_split(valid_dataset, [3000, 7000])

dataloader_valid = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=True
)

# [1] Kolesnikov, Alexander, et al. "Big transfer (bit): General visual representation learning." In Proc. of ECCV. 2020.

## ファインチューニングを行わない場合

まずは、ベースラインとして、ファインチューニングを行わない場合の精度を確認する。

モデルはResNet18を用いているが、別のモデルを用いてもよい。

In [ ]:
n_epochs = 10
lr = 0.001
device = 'cuda'

# pytorchではtorchvision.modelsから有名なモデルを呼び出してそのまま用いることが出来る
resnet18 = models.resnet18()

# デフォルトのresnet18は1000クラス分類用なので、全結合層を10クラス用に変更する
resnet18.fc = nn.Linear(512,10)

resnet18.to(device)
optimizer = optim.Adam(resnet18.parameters(), lr=lr)

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    resnet18.train()
    n_train = 0
    acc_train = 0
    for x, t in tqdm.notebook.tqdm(dataloader_train):
        n_train += t.size()[0]

        resnet18.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t = t.to(device)
        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = resnet18(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    resnet18.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t = t.to(device)
        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = resnet18(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

## 学習済みモデルの使用

次は学習済みモデルを用いる。

pytorchではtorchvisionのデフォルトのモデルに対して、ImageNetの学習済みモデルを用いることが出来る。

ImageNetは1000クラス約130万枚のデータセットで、CIFAR10よりもはるかに大規模なデータセットである。

In [ ]:
n_epochs = 10
lr = 0.001
device = 'cuda'

# pretrained = Trueと入れるとImageNetで事前学習された重みがセットされる
resnet18_pretrained = models.resnet18(pretrained=True)

resnet18_pretrained.fc = nn.Linear(512,10)

resnet18_pretrained.to(device)
optimizer = optim.Adam(resnet18_pretrained.parameters(), lr=lr)

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    resnet18_pretrained.train()
    n_train = 0
    acc_train = 0
    for x, t in tqdm.notebook.tqdm(dataloader_train):
        n_train += t.size()[0]

        resnet18_pretrained.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t = t.to(device)
        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = resnet18_pretrained(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    resnet18_pretrained.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t = t.to(device)
        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = resnet18_pretrained(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

デフォルトの設定では、10エポック学習したとき、ファインチューニング無しのモデルでは50%程度の精度しか達成できていない。

一方でファインチューニングを用いた場合、80%程度の精度を達成している。

80%という精度は、10分の1の量のCIFAR10及びResNet18という条件では、しっかり学習を回しても簡単には達成できない精度なので、この精度を数分で達成できるファインチューニングの有効性が示されている。

## torchvision.modelの紹介
torchvision.modelには、今回用いたResNet18以外にも様々なモデルが用意されているので紹介する
詳細は (https://pytorch.org/vision/stable/models.html) を参考にしていただきたい

In [ ]:
# torchvisionには例として以下のモデルが実装されており（他にもたくさんある）
# 全てのモデルに対してImageNet Pretrainを用いることが出来る
resnet18 = models.resnet18()
alexnet = models.alexnet()
vgg16 = models.vgg16()
squeezenet = models.squeezenet1_0()
densenet = models.densenet161()
inception = models.inception_v3()
googlenet = models.googlenet()
shufflenet = models.shufflenet_v2_x1_0()
mobilenet_v2 = models.mobilenet_v2()
mobilenet_v3_large = models.mobilenet_v3_large()
mobilenet_v3_small = models.mobilenet_v3_small()
resnext50_32x4d = models.resnext50_32x4d()
wide_resnet50_2 = models.wide_resnet50_2()
mnasnet = models.mnasnet1_0()